<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1280, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1280 (delta 41), reused 36 (delta 19), pack-reused 1212 (from 2)
Receiving objects: 100% (1280/1280), 13.30 MiB | 31.09 MiB/s, done.
Resolving deltas: 100% (829/829), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_stage2
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import trainer, trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 2) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
2.06% complete (down: 23387.6 kB/s up: 678.2 kB/s peers: 10) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://academictorrents.com/announce.php)[1

## Model

In [6]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=3, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<All keys matched successfully>

In [7]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model.load_state_dict({
    k: v for k, v in model_stage_1.state_dict().items() if "out" not in k
}, strict=False)

_IncompatibleKeys(missing_keys=['out.conv.weight', 'out.conv.bias'], unexpected_keys=[])

## Load data

In [8]:
full_train_dataset, val_dataset = get_datasets_stage2(source_folder=config["source_folder_lits"], seed=123, fold_number=2, model_stage_1=model_stage_1)
print(len(full_train_dataset), len(val_dataset))

105 26


In [9]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=0)

## Training

In [10]:
criterion = TverskyLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [ ]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/100 1/105 loss: 0.8558 time 3.98s
Epoch 1/100 2/105 loss: 0.8579 time 2.86s
Epoch 1/100 3/105 loss: 0.8477 time 3.36s
Epoch 1/100 4/105 loss: 0.8408 time 2.08s
Epoch 1/100 5/105 loss: 0.8390 time 4.56s
Epoch 1/100 6/105 loss: 0.8436 time 6.81s
Epoch 1/100 7/105 loss: 0.8499 time 7.87s
Epoch 1/100 8/105 loss: 0.8489 time 5.85s
Epoch 1/100 9/105 loss: 0.8501 time 2.00s
Epoch 1/100 10/105 loss: 0.8468 time 5.40s
Epoch 1/100 11/105 loss: 0.8379 time 5.06s
Epoch 1/100 12/105 loss: 0.8352 time 1.47s
Epoch 1/100 13/105 loss: 0.8359 time 1.37s
Epoch 1/100 14/105 loss: 0.8288 time 6.54s
Epoch 1/100 15/105 loss: 0.8311 time 5.10s
Epoch 1/100 16/105 loss: 0.8267 time 2.41s
Epoch 1/100 17/105 loss: 0.8250 time 5.98s
Epoch 1/100 18/105 loss: 0.8257 time 8.47s
Epoch 1/100 19/105 loss: 0.8258 time 1.83s
Epoch 1/100 20/105 loss: 0.8280 time 6.01s
Epoch 1/100 21/105 loss: 0.8290 time 7.70s
Epoch 1/100 22/105 loss: 0.8291 time 2.14s
Epoch 1/100 23/105 loss: 0.8296 time 1.53s
Epoch 1/100 24/105 l

Final training epochs: 1/100 ---[loss: 0.7943] ---[time 457.13s]
INFO:kltn.utils.logger:Final training epochs: 1/100 ---[loss: 0.7943] ---[time 457.13s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/100 105/105 loss: 0.7943 time 5.93s


Val 1/100 1/26, Dice_Tumor: 0.000000, time 1.99s
INFO:kltn.utils.logger:Val 1/100 1/26, Dice_Tumor: 0.000000, time 1.99s
Val 1/100 2/26, Dice_Tumor: 0.083311, time 1.21s
INFO:kltn.utils.logger:Val 1/100 2/26, Dice_Tumor: 0.083311, time 1.21s
Val 1/100 3/26, Dice_Tumor: 0.000000, time 6.68s
INFO:kltn.utils.logger:Val 1/100 3/26, Dice_Tumor: 0.000000, time 6.68s
Val 1/100 4/26, Dice_Tumor: 0.061433, time 7.86s
INFO:kltn.utils.logger:Val 1/100 4/26, Dice_Tumor: 0.061433, time 7.86s
Val 1/100 5/26, Dice_Tumor: 0.123135, time 1.16s
INFO:kltn.utils.logger:Val 1/100 5/26, Dice_Tumor: 0.123135, time 1.16s
Val 1/100 6/26, Dice_Tumor: 0.035682, time 5.28s
INFO:kltn.utils.logger:Val 1/100 6/26, Dice_Tumor: 0.035682, time 5.28s
Val 1/100 7/26, Dice_Tumor: 0.009524, time 5.63s
INFO:kltn.utils.logger:Val 1/100 7/26, Dice_Tumor: 0.009524, time 5.63s
Val 1/100 8/26, Dice_Tumor: 0.003148, time 4.01s
INFO:kltn.utils.logger:Val 1/100 8/26, Dice_Tumor: 0.003148, time 4.01s
Val 1/100 9/26, Dice_Tumor: 0.00

...

Epoch 89/100 1/105 loss: 0.4436 time 6.53s
Epoch 89/100 2/105 loss: 0.3117 time 8.72s
Epoch 89/100 3/105 loss: 0.3246 time 5.11s
Epoch 89/100 4/105 loss: 0.4190 time 1.53s
Epoch 89/100 5/105 loss: 0.4752 time 1.26s
Epoch 89/100 6/105 loss: 0.4224 time 3.66s
Epoch 89/100 7/105 loss: 0.4623 time 3.15s
Epoch 89/100 8/105 loss: 0.4573 time 6.15s
Epoch 89/100 9/105 loss: 0.4320 time 1.70s
Epoch 89/100 10/105 loss: 0.4588 time 7.24s
Epoch 89/100 11/105 loss: 0.4327 time 1.38s
Epoch 89/100 12/105 loss: 0.4205 time 6.19s
Epoch 89/100 13/105 loss: 0.3985 time 8.10s
Epoch 89/100 14/105 loss: 0.4047 time 5.40s
Epoch 89/100 15/105 loss: 0.3889 time 5.78s
Epoch 89/100 16/105 loss: 0.3859 time 5.39s
Epoch 89/100 17/105 loss: 0.3800 time 1.58s
Epoch 89/100 18/105 loss: 0.3658 time 5.06s
Epoch 89/100 19/105 loss: 0.3548 time 4.52s
Epoch 89/100 20/105 loss: 0.3492 time 6.91s
Epoch 89/100 21/105 loss: 0.3395 time 1.37s
Epoch 89/100 22/105 loss: 0.3309 time 2.12s
Epoch 89/100 23/105 loss: 0.3249 time 8.1

Final training epochs: 89/100 ---[loss: 0.3187] ---[time 452.06s]
INFO:kltn.utils.logger:Final training epochs: 89/100 ---[loss: 0.3187] ---[time 452.06s]

********************Epoch 89 Validation********************
INFO:kltn.utils.logger:
********************Epoch 89 Validation********************


Epoch 89/100 105/105 loss: 0.3187 time 1.92s


Val 89/100 1/26, Dice_Tumor: 0.000000, time 1.45s
INFO:kltn.utils.logger:Val 89/100 1/26, Dice_Tumor: 0.000000, time 1.45s
Val 89/100 2/26, Dice_Tumor: 0.330270, time 1.26s
INFO:kltn.utils.logger:Val 89/100 2/26, Dice_Tumor: 0.330270, time 1.26s
Val 89/100 3/26, Dice_Tumor: 0.000000, time 7.09s
INFO:kltn.utils.logger:Val 89/100 3/26, Dice_Tumor: 0.000000, time 7.09s
Val 89/100 4/26, Dice_Tumor: 0.504507, time 6.88s
INFO:kltn.utils.logger:Val 89/100 4/26, Dice_Tumor: 0.504507, time 6.88s
Val 89/100 5/26, Dice_Tumor: 0.331440, time 1.11s
INFO:kltn.utils.logger:Val 89/100 5/26, Dice_Tumor: 0.331440, time 1.11s
Val 89/100 6/26, Dice_Tumor: 0.355981, time 5.49s
INFO:kltn.utils.logger:Val 89/100 6/26, Dice_Tumor: 0.355981, time 5.49s
Val 89/100 7/26, Dice_Tumor: 0.055565, time 5.01s
INFO:kltn.utils.logger:Val 89/100 7/26, Dice_Tumor: 0.055565, time 5.01s
Val 89/100 8/26, Dice_Tumor: 0.395480, time 4.26s
INFO:kltn.utils.logger:Val 89/100 8/26, Dice_Tumor: 0.395480, time 4.26s
Val 89/100 9/26,

Epoch 90/100 1/105 loss: 0.1637 time 3.31s
Epoch 90/100 2/105 loss: 0.2206 time 6.39s
Epoch 90/100 3/105 loss: 0.2050 time 6.26s
Epoch 90/100 4/105 loss: 0.1891 time 2.39s
Epoch 90/100 5/105 loss: 0.1721 time 8.27s
Epoch 90/100 6/105 loss: 0.1798 time 6.65s
Epoch 90/100 7/105 loss: 0.1768 time 2.01s
Epoch 90/100 8/105 loss: 0.2422 time 7.45s
Epoch 90/100 9/105 loss: 0.2326 time 1.61s
Epoch 90/100 10/105 loss: 0.2333 time 7.38s
Epoch 90/100 11/105 loss: 0.2221 time 4.66s
Epoch 90/100 12/105 loss: 0.2201 time 5.57s
Epoch 90/100 13/105 loss: 0.2585 time 4.86s
Epoch 90/100 14/105 loss: 0.2565 time 8.75s
Epoch 90/100 15/105 loss: 0.2475 time 2.10s
Epoch 90/100 16/105 loss: 0.2430 time 1.29s
Epoch 90/100 17/105 loss: 0.2699 time 6.20s
Epoch 90/100 18/105 loss: 0.2623 time 1.92s
Epoch 90/100 19/105 loss: 0.2860 time 1.59s
Epoch 90/100 20/105 loss: 0.2815 time 4.59s
Epoch 90/100 21/105 loss: 0.2901 time 5.64s
Epoch 90/100 22/105 loss: 0.2904 time 1.12s
Epoch 90/100 23/105 loss: 0.2844 time 1.6

In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)